# Total Orders/Items Forecasting (Synthetic Demo)

This notebook is cleaned for GitHub and uses **synthetic sample data**.

## How to run
1) `pip install -r requirements.txt`
2) Run all cells


In [ ]:
import pandas as pd

DATA_PATH = "../examples/sample_total.csv"
df = pd.read_csv(DATA_PATH, parse_dates=["date"]) 
df.head()

In [ ]:
import numpy as np
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import plotly.express as px
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from pycaret.regression import *
from sklearn.ensemble import ExtraTreesRegressor


In [ ]:
indexes_to_drop = range(2072, 2102)  # EXclude azar

# Drop the specified rows
df = df.drop(index=indexes_to_drop)

# If you want to reset the index after dropping, use:
df = df.reset_index(drop=True)

In [ ]:
#considering campgain codes as input too
# Split the data into training and testing sets
#also y scaled
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:-383].drop(columns=['order_count','persian_date','organic_order',]),
                                                    df.iloc[:-383]['order_count'], test_size=0.2, random_state=42)

In [ ]:
exp = setup(data=pd.concat([X_train, y_train], axis=1), target='order_count', ignore_features=['persian_date','organic_order'], session_id=123)

# Compare models and select Extra Trees Regressor
best_model = compare_models()
etr = create_model('et')



In [ ]:
import optuna
from sklearn.metrics import mean_absolute_error

def objective(trial):

  reg = ExtraTreesRegressor(max_depth=trial.suggest_int("max_depth", 5, 100))
  reg.fit(X_train, y_train)

  y_pred = reg.predict(X_test)

  return mean_absolute_error(y_test, y_pred)

In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)

In [ ]:
study.best_params
et = ExtraTreesRegressor(**study.best_params)
et.fit(X_train, y_train)
y_pred = et.predict(X_test)


In [ ]:
for name, value in sorted(zip(X_train.columns, et.feature_importances_), key=lambda x: x[1], reverse=True):
    print(name, value)


In [ ]:
import matplotlib.pyplot as plt

plt.plot(y_test.values[0:100], "b")
plt.plot(y_pred[0:100], "r--")
plt.show()

In [ ]:
plt.plot(y_test.values - y_pred)
plt.show()

In [ ]:
x=df.iloc[-383:].drop(columns=['order_count','persian_date','organic_order'])
y=df.iloc[-383:]['order_count']

In [ ]:
y_pred=et.predict(x)

In [ ]:
out=pd.DataFrame()
out['date']=df.iloc[-383:]['persian_date']
out['yhat']=y_pred
out['y']=y

In [ ]:
out.to_csv('order_total_till_esf1404_withoutazar2-4.csv')

In [ ]:
import pickle

try:
    # Save the model to a file
    with open('model_et_order_danesh.pkl', 'wb') as file:
        pickle.dump(et, file)
    print("Model saved successfully!")
except Exception as e:
    print(f"An error occurred: {e}")
